## about this notebook:
- the goal is to compare a small sequence against a long sequence, and see how well they match at each position.
- here it is all about how to make this comparison as fast as possible

In [1]:
import numpy as np
from numba import njit

### testing on numeric sequences

In [2]:
ref = np.random.randint(low = 0, high = 4, size = 1000) # long reference sequence
seq = ref[:10] # short target sequence

In [3]:
def sc_py(ref, seq):
    cv = np.zeros(len(ref)-len(seq)+1, dtype = "int8")
    for i, _ in enumerate(cv):
        cv[i] = np.sum(ref[i:i+len(seq)] == seq)
    return cv
    
cv_py = sc_py(ref, seq)    
%timeit -r 100 -n 10 sc_py( ref,  seq)

3.17 ms ± 31.9 μs per loop (mean ± std. dev. of 100 runs, 10 loops each)


In [4]:
def encoder(seq):
    array = np.zeros((len(seq),4))
    for i in range(len(array)):
        array[i, seq[i]] = 1
    return array
    
def sc_np(ref, seq): # this allows not only for counting mismatches, but would also allow for giving them different weights
    REF = encoder(ref)
    SEQ = encoder(seq)
    cv = np.correlate(REF.flatten(), SEQ.flatten(), mode = "valid")[::4]
    return cv
    
cv_np = sc_np(ref, seq)    
%timeit -r 100 -n 10 sc_np( ref,  seq)
assert np.all(cv_py == cv_np)

215 μs ± 2.82 μs per loop (mean ± std. dev. of 100 runs, 10 loops each)


In [5]:
@njit 
def sc_jit(ref, seq):
    cv = np.zeros(len(ref)-len(seq)+1, dtype = "int8")
    for i, _ in enumerate(cv):
        cv[i] = np.sum(np.equal(ref[i:i+len(seq)], seq))
    return cv
    
cv_jit = sc_jit(ref, seq);

%timeit -r 100 -n 100 sc_jit( ref,  seq) # does only work with numbers
del sc_jit
assert np.all(cv_py == cv_jit)

27.8 μs ± 214 ns per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [6]:
@njit 
def sc_jit(ref, seq):
    n = len(ref)-len(seq)+1
    cv = [0]*n
    for i in range(n):
        count = 0
        for j in range(len(seq)):
            if ref[i+j] == seq[j]:
                count += 1
        cv[i] = count
    return cv
    
cv_jit = sc_jit(ref, seq);
%timeit -r 100 -n 10 sc_jit( ref,  seq)
del sc_jit
assert np.all(cv_py == cv_jit)

12.8 μs ± 392 ns per loop (mean ± std. dev. of 100 runs, 10 loops each)


In [7]:
@njit
def sc_jit(ref, seq):
    n = len(ref) - len(seq) + 1
    cv = np.zeros(n, dtype=np.int8) 
    for i in range(n):
        count = 0
        for j in range(len(seq)):
            if ref[i + j] == seq[j]:
                count += 1
        cv[i] = count
    return cv
    
cv_jit = sc_jit( ref,  seq)
%timeit -r 100 -n 1000 sc_jit(ref, seq)
assert np.all(cv_py == cv_jit)

4.83 μs ± 102 ns per loop (mean ± std. dev. of 100 runs, 1,000 loops each)


### But sequences come as strings

In [8]:
_ref = "".join(np.array(["A", "C", "G", "T"])[ref])
_seq = "".join(np.array(["A", "C", "G", "T"])[seq])

In [9]:
cv_jit = sc_jit( _ref,  _seq)
%timeit -r 100 -n 100 sc_jit(_ref, _seq)
assert np.all(cv_py == cv_jit)

268 μs ± 2.12 μs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [10]:
cv_jit = sc_jit(np.array([int(char) for char in _ref.replace("A", "0").replace("C", "1").replace("G", "2").replace("T", "3")]), np.array([int(char) for char in _seq.replace("A", "0").replace("C", "1").replace("G", "2").replace("T", "3")]))
%timeit -r 100 -n 100 sc_jit(np.array([int(char) for char in _ref.replace("A", "0").replace("C", "1").replace("G", "2").replace("T", "3")]), np.array([int(char) for char in _seq.replace("A", "0").replace("C", "1").replace("G", "2").replace("T", "3")]))
assert np.all(cv_py == cv_jit)

89.8 μs ± 1.13 μs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [11]:
sc_jit(_ref.encode('utf-8'), _seq.encode('utf-8'))
%timeit  -r 100 -n 1000 sc_jit(_ref.encode('utf-8'), _seq.encode('utf-8'))
assert np.all(cv_py == cv_jit)

4.99 μs ± 40.5 ns per loop (mean ± std. dev. of 100 runs, 1,000 loops each)


In [22]:
%timeit -r 100 -n 1000000 _ref.encode('utf-8')

80.6 ns ± 1.34 ns per loop (mean ± std. dev. of 100 runs, 1,000,000 loops each)


In [23]:
%timeit -r 100 -n 1000000 _ref.encode('ascii')

80.8 ns ± 1.23 ns per loop (mean ± std. dev. of 100 runs, 1,000,000 loops each)
